# Imports and Installs

In [0]:
# install latest version of polyglot language detection library for stretch
!pip install -U git+https://github.com/aboSamoor/polyglot.git@master

In [0]:
import pandas as pd
import numpy as np 
import pandas_profiling 
from sklearn import preprocessing # for category encoder
from polyglot.detect import Detector # for language detection stretch
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
# much more efficient for larger files like Nearest Neighbors which the model
import joblib

# EDA & PROFILE REPORT

In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/aguilargallardo/DS-Unit-2-Applied-Modeling/master/data/SpotifyFeatures.csv')

In [0]:
pandas_profiling.ProfileReport(df)

# Data Engineering Language Detection (Stretch, not working yet)

In [0]:
# test of polyglot detector

arabic_text = u"""
أفاد مصدر امني في قيادة عمليات صلاح الدين في العراق بأن " القوات الامنية تتوقف لليوم
الثالث على التوالي عن التقدم الى داخل مدينة تكريت بسبب
انتشار قناصي التنظيم الذي يطلق على نفسه اسم "الدولة الاسلامية" والعبوات الناسفة
والمنازل المفخخة والانتحاريين، فضلا عن ان القوات الامنية تنتظر وصول تعزيزات اضافية ".
"""

detector = Detector(arabic_text)
print(detector.language)

In [0]:
"""
   TODO: figure out UTF-8 ERROR or just do try catch exception and return 
   unknown or something
   below code does not work
"""

# df.track_name = df.track_name.astype(str)
# df['poly_obj'] = df.track_name.apply(lambda x: Detector(x, quiet=True))
# df['Track-lang'] = df['poly_obj'].apply(lambda x: icu.Locale.getDisplayName(x.language.locale))
# df['Track-LangConfidence'] = df['poly_obj'].apply( lambda x: x.language.confidence)


# DATA ENGINEERING CATEGORY ENCODING OF OBJECT FEATURES
note: might need to create another column of encoded languages
resources below: https://chrisalbon.com/machine_learning/preprocessing_structured_data/convert_pandas_categorical_column_into_integers_for_scikit-learn/


In [0]:
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    return(res)

In [0]:
df = encode_and_bind(df, 'key')
df = encode_and_bind(df, 'mode')
df = encode_and_bind(df, 'time_signature')


In [0]:
# check worked out
df.dtypes

# MODELING: Nearest Neighbors
resources: https://scikit-learn.org/stable/modules/neighbors.html

In [0]:
neigh = NearestNeighbors()

In [0]:
# to remove the transformed columns from model 
remove = ['key', 'mode','time_signature']
features = [i for i in list(df.columns[4:]) if i not in remove]
# target = 'track_id'

In [0]:
X = df[features]
# y = df[target]

X.shape, #y.shape

In [0]:
neigh.fit(X) # NN doesn't need to fit Y

#### Nicole's Imported Model

In [0]:
# K nearest neighbors NN https://www.reddit.com/r/MachineLearning/comments/2f8jff/using_neural_networks_for_nearest_neighbor/

In [0]:
rom knn_from_scratch import knn, euclidean_distance

def recommend_songs(movie_query, k_recommendations):
    raw_movies_data = []
    with open('movies_recommendation_data.csv', 'r') as md:
        # Discard the first line (headings)
        next(md)

        # Read the data into memory
        for line in md.readlines():
            data_row = line.strip().split(',')
            raw_movies_data.append(data_row)

    # Prepare the data for use in the knn algorithm by picking
    # the relevant columns and converting the numeric columns
    # to numbers since they were read in as strings
    songs_recommendation_data = []
    for row in raw_movies_data:
        data_row = list(map(float, row[2:]))
        songs_recommendation_data.append(data_row)

    # Use the KNN algorithm to get the 5 movies that are most
    # similar to The Post.
    recommendation_indices, _ = knn(
        movies_recommendation_data, movie_query, k=k_recommendations,
        distance_fn=euclidean_distance, choice_fn=lambda x: None
    )

    movie_recommendations = []
    for _, index in recommendation_indices:
        movie_recommendations.append(raw_movies_data[index])

    return movie_recommendations

if __name__ == '__main__':
    the_post = [7.2, 1, 1, 0, 0, 0, 0, 1, 0] # feature vector for The Post
    recommended_movies = recommend_movies(movie_query=the_post, k_recommendations=5)

    # Print recommended song titles
    for recommendation in recommended_songs:
        print(recommendation[10])

# Export Model with Joblib

In [0]:
filename = 'NearestNeighbor.sav'

In [0]:
joblib.dump(neigh, filename)